In [3]:
import os
from pathlib import Path
from googletrans import Translator
import time

In [6]:
# Translate Hebrew to English using Google Translate
def has_hebrew(text):
    """Check if text contains Hebrew characters"""
    return any('\u0590' <= char <= '\u05FF' for char in text)

def translate_hebrew_to_english(text, translator):
    """Translate Hebrew text to English"""
    try:
        # Remove file extension if exists for better translation
        translation = translator.translate(text, src='iw', dest='en')
        result = translation.text
        # Replace spaces with underscores and clean special characters
        result = result.replace(' ', '_')
        result = ''.join(c for c in result if c.isalnum() or c in '._-')
        return result
    except Exception as e:
        print(f"Translation error for '{text}': {e}")
        return text

def rename_directory_tree_with_translation(root_path):
    """Rename all files and folders from Hebrew to English with translation and numbering"""
    root_path = Path(root_path)
    
    if not root_path.exists():
        print(f"Error: Path does not exist: {root_path}")
        return
    
    translator = Translator()
    
    # Collect all paths (process deepest first to avoid path conflicts)
    all_paths = []
    for dirpath, dirnames, filenames in os.walk(root_path, topdown=False):
        current_dir = Path(dirpath)
        
        # Add files
        for filename in filenames:
            file_path = current_dir / filename
            all_paths.append(('file', file_path))
        
        # Add directories
        for dirname in dirnames:
            dir_path = current_dir / dirname
            all_paths.append(('dir', dir_path))
    
    # Rename all items with numbering
    renamed_count = 0
    skipped_count = 0
    item_counter = 1
    
    print(f"Found {len(all_paths)} items to process\n")
    
    for item_type, old_path in all_paths:
        old_name = old_path.name
        
        # Split name and extension
        if item_type == 'file' and '.' in old_name:
            name_part, ext_part = old_name.rsplit('.', 1)
            
            # Translate if has Hebrew, otherwise keep original
            if has_hebrew(name_part):
                translated_name = translate_hebrew_to_english(name_part, translator)
                new_name = f"{item_counter:03d}_{translated_name}.{ext_part}"
                time.sleep(0.1)  # Delay to avoid API rate limits
            else:
                new_name = f"{item_counter:03d}_{name_part}.{ext_part}"
        else:
            # Translate if has Hebrew, otherwise keep original
            if has_hebrew(old_name):
                translated_name = translate_hebrew_to_english(old_name, translator)
                new_name = f"{item_counter:03d}_{translated_name}"
                time.sleep(0.1)  # Delay to avoid API rate limits
            else:
                new_name = f"{item_counter:03d}_{old_name}"
        
        # Clean up multiple underscores
        while '__' in new_name:
            new_name = new_name.replace('__', '_')
        new_name = new_name.strip('_')
        
        # Ensure name is not empty
        if not new_name or new_name.startswith('.'):
            new_name = f"{item_counter:03d}_renamed_item"
            if item_type == 'file' and '.' in old_name:
                new_name += '.' + old_name.rsplit('.', 1)[1]
        
        new_path = old_path.parent / new_name
        
        # Check if new name already exists
        if new_path.exists() and new_path != old_path:
            counter = 1
            base_name = new_name
            if item_type == 'file' and '.' in new_name:
                base_name, ext = new_name.rsplit('.', 1)
                while new_path.exists():
                    new_name = f"{base_name}_dup{counter}.{ext}"
                    new_path = old_path.parent / new_name
                    counter += 1
            else:
                while new_path.exists():
                    new_name = f"{base_name}_dup{counter}"
                    new_path = old_path.parent / new_name
                    counter += 1
        
        try:
            if new_path != old_path:
                old_path.rename(new_path)
                print(f"✓ [{item_counter:03d}] [{item_type}] {old_name} → {new_name}")
                renamed_count += 1
            else:
                print(f"- [{item_counter:03d}] [{item_type}] {old_name} (no change needed)")
        except Exception as e:
            print(f"✗ [{item_counter:03d}] Failed to rename {old_name}: {e}")
            skipped_count += 1
        
        item_counter += 1
    
    print(f"\n{'='*50}")
    print(f"Processed: {item_counter - 1} items")
    print(f"Renamed: {renamed_count} items")
    print(f"Skipped: {skipped_count} items")
    print(f"{'='*50}")


In [7]:
# Set your directory path here
directory_path = r"C:\Users\thaim\Videos\AI_LEDS"

# Run the translator with numbering
rename_directory_tree_with_translation(directory_path)


Found 132 items to process

✓ [001] [file] 40_mtr_rka_la_shchvr_3_ldym_shmalh_1.mp4 → 001_40_mtr_rka_la_shchvr_3_ldym_shmalh_1.mp4
✓ [002] [file] rka_shchvr_lglay_40_mtr_shmalh_3_ldym_1.mp4 → 002_rka_shchvr_lglay_40_mtr_shmalh_3_ldym_1.mp4
✓ [003] [dir] 40_mtr_rka_la_shchvr_3_ldym_shmalh → 003_40_mtr_rka_la_shchvr_3_ldym_shmalh
✓ [004] [dir] rka_shchvr_lglay_40_mtr_shmalh_3_ldym → 004_rka_shchvr_lglay_40_mtr_shmalh_3_ldym
✓ [005] [file] 30_rgyl_ayn_shmsh_ldym_mkdymh_rka_shkvf.mp4 → 005_30_rgyl_ayn_shmsh_ldym_mkdymh_rka_shkvf.mp4
✓ [006] [file] 30_zvvyt_ymynh_ayn_shmsh_ldym_mkdymh_rka_shkvf.mp4 → 006_30_zvvyt_ymynh_ayn_shmsh_ldym_mkdymh_rka_shkvf.mp4
✓ [007] [file] 30_zvvyvt_lmth_lashvt_arykh_lsrtvn.mp4 → 007_30_zvvyvt_lmth_lashvt_arykh_lsrtvn.mp4
✓ [008] [file] 40_lmalh_zvvyt_ymynh.mp4 → 008_40_lmalh_zvvyt_ymynh.mp4
✓ [009] [file] 40_rtz.mp4 → 009_40_rtz.mp4
✓ [010] [file] 40_ymynh_lmth.mp4 → 010_40_ymynh_lmth.mp4
✓ [011] [file] 50_zvvyt_ymynh.mp4 → 011_50_zvvyt_ymynh.mp4
✓ [012] [file